
## Analisis Red Social

In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import glob
import sys
import pandas as pd
#import tensorflow
#import keras
import wordcloud
import nltk
import inspect
import re
import unicodedata

from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import RSLPStemmer #Stemmer for portugese words.

from nltk.probability import FreqDist
from collections import defaultdict
from heapq import nlargest

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.gridspec import GridSpec
#import missingno as msno
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
import string
import sys
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.sparse import csr_matrix
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score,roc_auc_score,roc_curve
from sklearn.decomposition import PCA

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression


## Aca leo la base de datos a usar con las variables

In [3]:
os.chdir("C:\\Users\\d.cadavid141\\Documents\\Coisas Daniela\\201920\\201920\\00_Bases\\brazilian-ecommerce\\")

In [4]:
## Variables
list_vars=pd.read_csv('feature_imp_100vars.csv')
vars_usar=list_vars[list_vars.columns[0]][:139]
v_random =list_vars[list_vars.columns[0]][:11]
v_lr=list_vars[list_vars.columns[0]][:652]

## Base redes
orders=pd.read_csv('data_tratada_manual_v2.csv')

## Base vacia
rows=orders.shape[0]
orig_cols_lr=list(vars_usar)
orig_cols_rf=list(v_random)
orig_col_lr=list(v_lr)
zeros=pd.DataFrame(np.zeros((rows,len(orig_cols_lr))),columns=orig_cols_lr)
zeros_rf=pd.DataFrame(np.zeros((rows,len(orig_cols_rf))),columns=orig_cols_rf)
zeros_lr=pd.DataFrame(np.zeros((rows,len(orig_col_lr))),columns=orig_col_lr)

In [5]:
base_red=pd.DataFrame(orders)
base_red.shape

(668, 4)

## Generar los data sets

In [6]:
def generate_zeros_dataset(orig_df, other_df):
    rows = other_df.shape[0]
    orig_colnames = list(orig_df.columns)
    dtypes = dict(orig_df.dtypes)
    d = pd.DataFrame(np.zeros((rows, len(orig_colnames))), columns=orig_colnames)
    return d.astype(dtypes) # force the same datatypes =-)

# grab both dataset columns
def generate_dataset(orig_df, other_df):
    colnames_2 = list(other_df.columns)
    d = generate_zeros_dataset(orig_df, other_df)
    d[colnames_2] = other_df.copy()
    return d

## CREO MI CONJUNTO DE DATOS

In [7]:
####### ACA SE CREA EL TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(orders.Comment).toarray()
X_val_mod=pd.DataFrame(X_tfidf)
y_val=orders.Label
cols_manual=vectorizer.get_feature_names()
X_val_mod.columns = cols_manual
X_val_mod.head()

,abaixar,abrir,absurdo,acabamento,acabar,aceitar,acender,acessorio,acessorios,achar,...,vivar,voltar,volumar,voz,whatsapp,xiaomi,ximo,youtube,zap,zero
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Boosting

In [8]:
X_val_mod=X_val_mod[X_val_mod.columns.intersection(vars_usar)]

reg_manual = generate_dataset(zeros, X_val_mod)

In [9]:
vars_usar=139

In [10]:
import pickle

In [11]:
# load the model from disk
filename='finalized_model_rfim_gb.sav'
loaded_model = pickle.load(open(filename, 'rb'))
gb_roc_auc=roc_auc_score(y_val, loaded_model.predict(reg_manual))

In [13]:
gb_roc_auc

0.7991401054803945

In [14]:
gb_f1_test = f1_score(y_val, loaded_model.predict(reg_manual))
gb_f1_test

0.8467475192943771

## Reg Logistica 

In [15]:
vars_usar=652

In [16]:
X_val_mod=X_val_mod[X_val_mod.columns.intersection(v_lr)]

lr_manual = generate_dataset(zeros_lr, X_val_mod)

In [17]:
# load the model from disk
filename='finalized_model_rfim_lr.sav'
loaded_model = pickle.load(open(filename, 'rb'))
lr_roc_auc=roc_auc_score(y_val, loaded_model.predict(lr_manual))

In [19]:
lr_roc_auc

0.7343040587021326

In [20]:
lr_f1_test = f1_score(y_val, loaded_model.predict(lr_manual))
lr_f1_test

0.8013392857142858

## Random Forest

In [21]:
vars_usar=11

In [22]:
X_val_mod=X_val_mod[X_val_mod.columns.intersection(v_random)]

rf_manual = generate_dataset(zeros_rf, X_val_mod)

In [23]:
# load the model from disk
filename='finalized_model_rfim_rf.sav'
loaded_model = pickle.load(open(filename, 'rb'))
rf_roc_auc=roc_auc_score(y_val, loaded_model.predict(rf_manual))

In [24]:
rf_roc_auc

0.612646182068333

In [25]:
rf_f1_test = f1_score(y_val, loaded_model.predict(rf_manual))
rf_f1_test

0.8542246982358404